In [ ]:
from transformers import pipeline

# Tokenizer

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier(["I have been waiting for this all my life"])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9726982712745667}]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
raw_inputs = ["I am loving this course, I dislike this world", "i am gonna take this horse to the graveyard "]

In [ ]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1045,  2572,  8295,  2023,  2607,  1010,  1045, 18959,  2023,
          2088,   102],
        [  101,  1045,  2572,  6069,  2202,  2023,  3586,  2000,  1996, 16685,
           102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [ ]:
"""
the autotokenizer can load the tokenizer for any checkpoint,
padding to be done as sentences not the same size, truncate if size greater than max defined mask, return pytorch tensors
attention mask details where padding has been applued



downlaod and caache the configuration of the model as well as pretrained weights
AutoModel will only instantiate the body of the model, that is part of the model that is left onice pretraining head is removed, will output a high deimnesiobal tensor which is a
representation of the sentences passed, whic is not directly usefull for our classification problem

torch.Size([2, 12, 768]) representation of the sentences passed,
(batch size, sequence length,, hidden size)

 """

SyntaxError: incomplete input (<ipython-input-16-aa1c9c5d96c6>, line 1)

In [ ]:
from transformers import AutoModel
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
output = model(**inputs)
print(output.last_hidden_state.shape)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

torch.Size([2, 12, 768])


In [ ]:
"""

AutoModelForSequenceClassification for classification, exceprt will build a model with a classification head

we get matrix of 2x2, one result of each sentence and for each label, these outputs are not probabilities yet, as each model of the transformers library returns logits,
to make sense of the logits we need to make sense of the third adn last step pf the pipeline, which is post ptocrssing

convert logits to probabilities, we need to apply softmaxx layers to it

first sentence  N P
second sentence N P
"""

# Model

In [ ]:
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

output = model(**inputs)
print(output.logits)

tensor([[-3.1529,  3.3831],
        [ 2.2068, -2.0073]], grad_fn=<AddmmBackward0>)


# PostProcessing

In [ ]:
import torch
predictions = torch.nn.functional.softmax(output.logits, dim=1)
print(predictions)

tensor([[0.0014, 0.9986],
        [0.9854, 0.0146]], grad_fn=<SoftmaxBackward0>)


In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}